Importing dependencies

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import svm
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
import seaborn as sns
from sklearn.metrics import f1_score
%matplotlib inline
from sklearn import tree
from sklearn.tree import DecisionTreeRegressor


In [2]:
! pip install fast_ml --quiet

In [3]:
from fast_ml.model_development import train_valid_test_split

Resding the dataset with pandas

In [4]:
df= pd.read_excel('Project.xlsx')

In [5]:
df.head()

,Timestamp,FIT101,LIT101,MV101,P101,P102,AIT201,AIT202,AIT203,FIT201,...,P501,P502,PIT501,PIT502,PIT503,FIT601,P601,P602,P603,Normal/Attack
0,28/12/2015 10:00:00 AM,2.427057,522.8467,2,2,1,262.0161,8.396437,328.6337,2.445391,...,2,1,250.8652,1.649953,189.5988,0.000128,1,1,1,Normal
1,28/12/2015 10:00:01 AM,2.446274,522.8860,2,2,1,262.0161,8.396437,328.6337,2.445391,...,2,1,250.8652,1.649953,189.6789,0.000128,1,1,1,Normal
2,28/12/2015 10:00:02 AM,2.489191,522.8467,2,2,1,262.0161,8.394514,328.6337,2.442316,...,2,1,250.8812,1.649953,189.6789,0.000128,1,1,1,Normal
3,28/12/2015 10:00:03 AM,2.534350,522.9645,2,2,1,262.0161,8.394514,328.6337,2.442316,...,2,1,250.8812,1.649953,189.6148,0.000128,1,1,1,Normal
4,28/12/2015 10:00:04 AM,2.569260,523.4748,2,2,1,262.0161,8.394514,328.6337,2.443085,...,2,1,250.8812,1.649953,189.5027,0.000128,1,1,1,Normal


In [6]:
df.shape

(449775, 53)

In [7]:
df.describe()

,FIT101,LIT101,MV101,P101,P102,AIT201,AIT202,AIT203,FIT201,MV201,...,FIT504,P501,P502,PIT501,PIT502,PIT503,FIT601,P601,P602,P603
count,449775.000000,449775.000000,449775.000000,449775.000000,449775.000000,449775.000000,449775.000000,449775.000000,449775.000000,449775.000000,...,449775.000000,449775.000000,449775.0,449775.000000,449775.000000,449775.000000,449775.000000,449775.0,449775.000000,449775.0
mean,1.714083,607.035321,1.665228,1.693473,1.006948,210.308648,8.528483,320.311652,1.703419,1.691824,...,0.283799,1.925838,1.0,232.168691,1.013434,174.710712,0.016553,1.0,1.009076,1.0
std,1.191815,125.319579,0.482363,0.461052,0.083064,35.157817,0.114825,16.623894,1.130069,0.470479,...,0.080936,0.262034,0.0,61.760149,0.412786,47.486941,0.159557,0.0,0.094833,0.0
min,0.000000,189.826300,0.000000,1.000000,1.000000,168.033800,6.000000,285.337100,0.000000,0.000000,...,0.000000,1.000000,1.0,9.468726,0.000000,3.140220,0.000000,1.0,1.000000,1.0
25%,0.000000,510.011100,1.000000,1.000000,1.000000,177.102000,8.441298,306.408600,0.000000,1.000000,...,0.306249,2.000000,1.0,248.029400,0.816967,186.698900,0.000000,1.0,1.000000,1.0
50%,2.477020,530.383200,2.000000,2.000000,1.000000,193.540100,8.551525,321.661100,2.446673,2.000000,...,0.306633,2.000000,1.0,249.295100,0.961138,187.724300,0.000000,1.0,1.000000,1.0
75%,2.577907,727.471400,2.000000,2.000000,1.000000,254.005400,8.610806,331.684200,2.454362,2.000000,...,0.308106,2.000000,1.0,250.881200,1.409669,189.118200,0.000128,1.0,1.000000,1.0
max,2.760145,925.032300,2.000000,2.000000,2.000000,267.719800,8.733210,384.465500,2.826899,2.000000,...,0.311693,2.000000,1.0,254.341800,1.970333,191.986000,1.802710,1.0,2.000000,1.0


Now we want to check the Target values

In [8]:
df['Normal/Attack'].value_counts()

Normal     395154
Attack      54584
A ttack        37
Name: Normal/Attack, dtype: int64

There are some tiping issues in the Target values. Let's fix it 

In [9]:
df["Normal/Attack"].replace('A ttack', 'Attack', inplace=True)

In [10]:
df['Normal/Attack'].value_counts()

Normal    395154
Attack     54621
Name: Normal/Attack, dtype: int64

In [11]:
df = df.drop(columns=' Timestamp', axis=1)

Some values in the dataset are useless! It's better to remove them :))

In [12]:
df = df.replace(0, np.nan).interpolate(method ='linear', limit_direction ='forward').fillna(0)

In [13]:
df.info()
df.value_counts()[0]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 449775 entries, 0 to 449774
Data columns (total 52 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   FIT101         449775 non-null  float64
 1   LIT101         449775 non-null  float64
 2    MV101         449775 non-null  float64
 3   P101           449775 non-null  int64  
 4   P102           449775 non-null  int64  
 5    AIT201        449775 non-null  float64
 6   AIT202         449775 non-null  float64
 7   AIT203         449775 non-null  float64
 8   FIT201         449775 non-null  float64
 9    MV201         449775 non-null  float64
 10   P201          449775 non-null  int64  
 11   P202          449775 non-null  int64  
 12  P203           449775 non-null  int64  
 13   P204          449775 non-null  int64  
 14  P205           449775 non-null  int64  
 15  P206           449775 non-null  int64  
 16  DPIT301        449775 non-null  float64
 17  FIT301         449775 non-nul

KeyError: 0

Now it's time to split the features and Target

In [14]:
X = df.drop(columns='Normal/Attack', axis=1)
y = df['Normal/Attack'].values

In [15]:
print(X)

          FIT101    LIT101   MV101  P101  P102    AIT201    AIT202    AIT203  \
0       2.427057  522.8467     2.0     2     1  262.0161  8.396437  328.6337   
1       2.446274  522.8860     2.0     2     1  262.0161  8.396437  328.6337   
2       2.489191  522.8467     2.0     2     1  262.0161  8.394514  328.6337   
3       2.534350  522.9645     2.0     2     1  262.0161  8.394514  328.6337   
4       2.569260  523.4748     2.0     2     1  262.0161  8.394514  328.6337   
...          ...       ...     ...   ...   ...       ...       ...       ...   
449770  2.559972  519.5495     2.0     2     1  168.0979  8.638683  301.9226   
449771  2.549082  520.4131     2.0     2     1  168.0979  8.638683  301.9226   
449772  2.531467  520.6878     2.0     2     1  168.0979  8.638683  301.9226   
449773  2.521218  520.7271     2.0     2     1  168.0979  8.638683  301.9226   
449774  2.501681  521.1196     2.0     2     1  168.0979  8.638683  301.9226   

          FIT201   MV201  ...    FIT504

In [16]:
print(y)

['Normal' 'Normal' 'Normal' ... 'Normal' 'Normal' 'Normal']


Let's split the data into train, test and validation set

In [17]:
X_train, y_train, X_valid, y_valid, X_test, y_test = train_valid_test_split(df, target = 'Normal/Attack',
                                                                            train_size = .6, valid_size=0.2, test_size=0.2)

In [18]:
print(X_train.shape, y_train.shape)
print(X_valid.shape, y_valid.shape)
print(X_test.shape, y_test.shape)

(269865, 51) (269865,)
(89955, 51) (89955,)
(89955, 51) (89955,)


Now it's time to standardize the data

In [19]:
scaler = StandardScaler()
scaler.fit(X_train,)
scaler.fit(X_valid)
scaler.fit(X_test)

StandardScaler()

In [20]:
standardized_data_train = scaler.transform(X_train)
standardized_data_valid = scaler.transform(X_valid)
standardized_data_test = scaler.transform(X_test)

In [21]:
X_train = standardized_data_train
X_valid = standardized_data_valid
X_test = standardized_data_test

In [22]:
print(X.shape, X_train.shape, X_test.shape)

(449775, 51) (269865, 51) (89955, 51)


Defining the model

In [23]:
classifier = SGDClassifier(max_iter=1000, tol=0.01)
classifier.fit(X_train, y_train)

SGDClassifier(tol=0.01)

Let's find th accuracy

In [24]:
X_test_prediction = classifier.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, y_test)
print('Accuracy score of Test data:', test_data_accuracy)

Accuracy score of Test data: 0.9685398254682897


The accuracy is good but let's tune the hyperparameters to see if it  gets better

In [34]:
X_new = X

In [36]:
scaler.fit(X_new)

StandardScaler()

In [37]:
X_new =scaler.transform(X_new)

K_fold cross validation seems like a good way to validate our model!

In [26]:
skf = KFold(n_splits= 10, random_state=42, shuffle = True)

In [38]:
df_split_object = skf.split(X_new, y)

The code below wants to show that K_fold randomly pick the train and test set and keeps changing it

In [39]:
my_split_data= next(df_split_object)
print('train indices are:', my_split_data[0][0:50])
print('test indices are:', my_split_data[1][1:50])

train indices are: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49
 50 51]
test indices are: [ 31  53  81  85  91 100 102 117 119 123 130 176 222 226 237 250 256 261
 270 285 288 289 302 305 317 319 327 332 333 355 356 362 369 371 380 391
 396 401 413 416 419 427 442 450 482 483 493 502 508]


In [29]:
predicted_y =[]
expected_y =[]

In [30]:
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics

In [31]:
scaler2 = MinMaxScaler()

We measure the accuracy, precision, recall and f1score inside the model_testing function 
so we recognize wich hyperparameters increase the function of our model

In [40]:
def model_testing(hp_loss, hp_alpha,hp_itr, hp_tol):
    for train_index, test_index in skf.split(X_new, y):
        x_train, x_test = X_new[train_index], X_new[test_index]
        y_train_2, y_test_2 = y[train_index], y[test_index]
    
        scaler2.fit(x_train)
        x_train= scaler2.transform(x_train)
        x_test= scaler2.transform(x_test)
    
        classifier2 = SGDClassifier(loss= hp_loss, alpha= hp_alpha, max_iter=hp_itr, tol=hp_tol)
        classifier2.fit(x_train, y_train_2)
    
        predicted_this_fold = classifier2.predict(x_test)
        precision = metrics.precision_score(y_test_2, predicted_this_fold, pos_label= 'Normal')
        recall = metrics.recall_score(y_test_2, predicted_this_fold,pos_label= 'Normal' )
        f1score = metrics.f1_score(y_test_2, predicted_this_fold, pos_label= 'Normal')
        accuracy = metrics.accuracy_score(y_test_2, predicted_this_fold)

        print(f"Precision from this fold is = {precision}")
        print(f"Recall from this fold is = {recall}")
        print(f"F1 Score from this fold is = {f1score}")
        print('accuracy from this fold is:'+accuracy.__str__())
    
        predicted_y.extend(predicted_this_fold)
        expected_y.extend(y_test_2)
    precision = metrics.precision_score(expected_y, predicted_y, pos_label= 'Normal')
    recall = metrics.recall_score(expected_y, predicted_y, pos_label= 'Normal')
    f1score = metrics.f1_score(expected_y, predicted_y, pos_label= 'Normal')
    accuracy = metrics.accuracy_score(expected_y, predicted_y)
    print(f"Precision from all fold is = {precision}")
    print(f"Recall from all fold is = {recall}")
    print(f"F1 Score from all fold is = {f1score}")
    print('accuracy from all fold is:'+accuracy.__str__())

In [41]:
model_testing('hinge',0.001,1000, 0.01)

Precision from this fold is = 0.9604075974610278
Recall from this fold is = 0.9991145069068461
F1 Score from this fold is = 0.9793787587575175
accuracy from this fold is:0.9630263684467962
Precision from this fold is = 0.9598364247115525
Recall from this fold is = 0.9988853987232749
F1 Score from this fold is = 0.9789716725837284
accuracy from this fold is:0.9623371426030504
Precision from this fold is = 0.9597520962449873
Recall from this fold is = 0.9992914441885771
F1 Score from this fold is = 0.9791227591678857
accuracy from this fold is:0.9625594735203877
Precision from this fold is = 0.9600681762843924
Recall from this fold is = 0.9992144142317732
F1 Score from this fold is = 0.9792502266208044
accuracy from this fold is:0.9628485037129263
Precision from this fold is = 0.9595507803004522
Recall from this fold is = 0.9992405832320778
F1 Score from this fold is = 0.9789935765482006
accuracy from this fold is:0.9623371426030504
Precision from this fold is = 0.9607738167660299
Recall

In [42]:
model_testing('log',0.001,1000, 0.01)

Precision from this fold is = 0.9523476667952179
Recall from this fold is = 0.9996205029600769
F1 Score from this fold is = 0.9754116572444269
accuracy from this fold is:0.9557116812663969
Precision from this fold is = 0.9513177247847997
Recall from this fold is = 0.9994426993616374
F1 Score from this fold is = 0.9747865940283388
accuracy from this fold is:0.9546222597714438
Precision from this fold is = 0.9524452589948876
Recall from this fold is = 0.9994685831414328
F1 Score from this fold is = 0.9753905044143977
accuracy from this fold is:0.9556894481746632
Precision from this fold is = 0.951393076830298
Recall from this fold is = 0.9994678289957173
F1 Score from this fold is = 0.974838103712492
accuracy from this fold is:0.9547334252301125
Precision from this fold is = 0.9515169040217837
Recall from this fold is = 0.9995696638315107
F1 Score from this fold is = 0.9749515450045061
accuracy from this fold is:0.9548890568722487
Precision from this fold is = 0.9534984789222077
Recall f

Looks like hinge is doing better. 
Let's tune other hyperparameters

In [43]:
model_testing('hinge',0.0001,1000, 0.001)

Precision from this fold is = 0.9639630841349676
Recall from this fold is = 0.9988868086828923
F1 Score from this fold is = 0.9811142587346553
accuracy from this fold is:0.9662057005647205
Precision from this fold is = 0.9635114988879928
Recall from this fold is = 0.9986827439456886
F1 Score from this fold is = 0.980781908872663
accuracy from this fold is:0.9656498732713771
Precision from this fold is = 0.9631129543791168
Recall from this fold is = 0.9990130829769466
F1 Score from this fold is = 0.9807345945073098
accuracy from this fold is:0.9655164747209747
Precision from this fold is = 0.9633675464320626
Recall from this fold is = 0.9989863409442234
F1 Score from this fold is = 0.980853684328386
accuracy from this fold is:0.9657832718217796
Precision from this fold is = 0.9632640109353642
Recall from this fold is = 0.998962130417173
F1 Score from this fold is = 0.9807883487424197
accuracy from this fold is:0.9656276401796434
Precision from this fold is = 0.9641871259313546
Recall fr

In [44]:
model_testing('hinge',0.0001,10000, 0.001)

Precision from this fold is = 0.9640336955194726
Recall from this fold is = 0.9988868086828923
F1 Score from this fold is = 0.9811508306308322
accuracy from this fold is:0.9662723998399217
Precision from this fold is = 0.9635114988879928
Recall from this fold is = 0.9986827439456886
F1 Score from this fold is = 0.980781908872663
accuracy from this fold is:0.9656498732713771
Precision from this fold is = 0.9632304501646944
Recall from this fold is = 0.9990130829769466
F1 Score from this fold is = 0.980795508186132
accuracy from this fold is:0.9656276401796434
Precision from this fold is = 0.9636501417815586
Recall from this fold is = 0.9989863409442234
F1 Score from this fold is = 0.9810001368686931
accuracy from this fold is:0.9660500689225844
Precision from this fold is = 0.9623009583262211
Recall from this fold is = 0.998962130417173
F1 Score from this fold is = 0.980288897445133
accuracy from this fold is:0.9647160834185602
Precision from this fold is = 0.9642106806078077
Recall fro